In [5]:
import yfinance as yf
from curve import *
from balancer import *
from gmx import *

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# To get new data, delete the existing file in 'eth' folder
def get_eth_price():
    eth_folder = 'eth'
    eth_file = os.path.join(eth_folder, 'eth_price_data.csv')

    # Check if the ETH price data file exists
    if not os.path.exists(eth_file):
        # If the file doesn't exist, download the ETH price data
        end_date = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
        eth_data = yf.download('ETH-USD', start='2020-01-01', end=end_date)
        eth_data.reset_index(inplace=True)
        eth_data = eth_data.rename(columns={'Date': 'timestamp', 'Adj Close': 'eth_price'})
        eth_data = eth_data[['timestamp', 'eth_price']]
        eth_data['timestamp'] = eth_data['timestamp'].dt.strftime('%Y-%m-%d')

        # Save the data to the eth folder
        if not os.path.exists(eth_folder):
            os.makedirs(eth_folder)
        eth_data.to_csv(eth_file, index=False)
    else:
        # If the file exists, load the data from the file
        eth_data = pd.read_csv(eth_file)

    return eth_data

crv_df = merge_crv_data()
gmx_df = merge_gmx_data()
bal_df = merge_bal_data()

eth_df = get_eth_price()

# Add token_name column to each DataFrame
crv_df['token_name'] = 'CRV'
gmx_df['token_name'] = 'GMX'
bal_df['token_name'] = 'BAL'

# Concatenate the DataFrames
combined_df = pd.concat([crv_df, gmx_df, bal_df])

# Merge with ETH price data
combined_df = pd.merge(combined_df, eth_df, on='timestamp', how='left')

# Print the combined DataFrame info to verify
print(combined_df.info())
print(combined_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   timestamp             2410 non-null   object 
 1   price                 2410 non-null   float64
 2   circ_supply           2410 non-null   float64
 3   bonded_supply         2410 non-null   float64
 4   bonded_percent        2410 non-null   float64
 5   total_supply          2410 non-null   float64
 6   daily_inflation_rate  2410 non-null   float64
 7   apr                   2410 non-null   float64
 8   has_liquid_staking    2410 non-null   bool   
 9   token_name            2410 non-null   object 
 10  eth_price             2410 non-null   float64
dtypes: bool(1), float64(8), object(2)
memory usage: 190.8+ KB
None
    timestamp  price   circ_supply  bonded_supply  bonded_percent  \
0  2020-09-08   2.09  4.027460e+07   5.172288e+06        0.113809   
1  2020-09-09   2.12  4.